In [ ]:
# Import the necessary libraries to perform the proper analysis
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

In [ ]:
# Create variables to store the host of the API and which dataset you would like ot use.
# For my purposes, I only used the Summary of Businesses dataset
HOST = "https://api.census.gov/data"
dataset1 = "2018/abscs?"

# May have to use an API key at the end of the URL, if so, obtain an API key from the website of the API
# Create a base url by combining the dataset and the HOST
base_url = "/".join([HOST, dataset1])

# Create a dictionary for the predicates you want to look at
predicates = {}

# Call our which variables you would like to pull from the dataset
# Sex, Sex_Label, Employee Count, and Total Payroll are included for this example
# Important to exactly match how they are listed in the dataset, check the dataset to make sure
get_vars = ["SEX","SEX_LABEL","EMP","PAYANN"]

# Fill in the predicates dictionary by joining the variables and attaching them to the get key
# Fill in the predicates dictionary by calling which level of data you want to look at and attaching them to the for key
# In this example we wanted to look at aggregated data for each state
# Could look at aggregated data for the entire US or individual states
predicates['get'] = ",".join(get_vars)
predicates['for'] = "state:*"
r = requests.get(base_url, params=predicates)

# Create a col_names list to change the column names to something that may be easier to work with
# In this example, I changed the column names to lower case and something that had more meaning to me than the given codes
# Despite the "state" column not being included in the get_vars list, the state code will be included as a column
col_names = ["sex_code","sex_label","employee_count","total_payroll","state"]

# Create a Pandas dataframe using the data that we took in from the API
# Set the columns equal to the col_names list and the data equal to a json of the API data you called
# Skip the first row of the json for the data because that includes the old column headers that we no longer need
df = pd.DataFrame(columns=col_names, data=r.json()[1:])

# Set the data types of the variables you pulled to be whatever you need them to be
df["sex_code"] = df["sex_code"].astype(int)
df["employee_count"] = df["employee_count"].astype(int)
df["total_payroll"] = df["total_payroll"].astype(float)
df["state"] = df["state"].astype(int)

# Can then use Seaborn, matplotlib, or plotly among others to make whatever visualizations are desired

# Seaborn regplot code example
sns.set_theme(style="whitegrid")
f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.despine(f, left=True, bottom=True)
fig = sns.regplot(x="employee_count", y="total_payroll",
                data=df, ax=ax)
fig.set(xlabel='Employee Count (Scientific Notation)', ylabel='Total Payroll (Scientific Notation)',
        title='Total Payroll in Each State based on the Employee Count')

# Matplotlib bar plot example
top_ten.plot.bar(x = 'state_name', y = 'pay/emp')
plt.title('States with the Highest Pay')
plt.xlabel('State')
plt.ylabel('Pay per Employee')
plt.show()

# Plotly grouped bar plot example
fig = px.bar(top_ten, x="state_name", y="pay/emp",color="sex_label",barmode='group',
             labels={
                     "state_name": "State",
                     "pay/emp": "Payroll per Employee",
                     "sex_label": "Gender"
                 }
             ,title='Top Ten Payrolls per Employee',height=400)
fig.show()

In [ ]:
# You may also want to create a new column of data to work with or only select certain portions of data

# To create a new column, you can do something such as dividing one column in a dataframe by another
categories['pay/emp'] = categories['total_payroll']/categories['employee_count']
# This new column could then be used in graphs created in the future

# Could also do something such as creating a new dataframe with sorted values
sorted_1 = categories.sort_values(by=['pay/emp'],ascending = False)

# Can then filter down to only certain values in the list
top_ten = sorted_1[0:10]

# Can filter datasets to only include certain values in the column
# In this example, I filter out the values in the gender column that aren't male or female
# Can be valuable for this dataset as it includes total values for certain columns
dropped_totals = categories[(categories['sex_label'] == 'Male') | (categories['sex_label'] == 'Female')]

# Could also merge this dataset with another by creating two dataframes and merging them based on a shared column
# For my specific purposes I did not have to use this method
# Please reference my team members ETL documents for information regarding merging